In [41]:
import pandas as pd
df = pd.read_csv("/content/100_Unique_QA_Dataset.csv")

In [42]:
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [43]:
# tokenize the words
def tokenize(text):
  text = text.lower()
  text = text.replace("?", "")
  text = text.replace("'", "")
  return text.split()

In [44]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [45]:
# vocabulary
vocab = {'<UNK>':0}

In [46]:
def build_vocab(row):
  print(row['question'], row['answer'])
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  #merge both into single list
  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)

  print(merged_tokens)

In [47]:
df.apply(build_vocab, axis=1)

What is the capital of France? Paris
['what', 'is', 'the', 'capital', 'of', 'france', 'paris']
What is the capital of Germany? Berlin
['what', 'is', 'the', 'capital', 'of', 'germany', 'berlin']
Who wrote 'To Kill a Mockingbird'? Harper-Lee
['who', 'wrote', 'to', 'kill', 'a', 'mockingbird', 'harper-lee']
What is the largest planet in our solar system? Jupiter
['what', 'is', 'the', 'largest', 'planet', 'in', 'our', 'solar', 'system', 'jupiter']
What is the boiling point of water in Celsius? 100
['what', 'is', 'the', 'boiling', 'point', 'of', 'water', 'in', 'celsius', '100']
Who painted the Mona Lisa? Leonardo-da-Vinci
['who', 'painted', 'the', 'mona', 'lisa', 'leonardo-da-vinci']
What is the square root of 64? 8
['what', 'is', 'the', 'square', 'root', 'of', '64', '8']
What is the chemical symbol for gold? Au
['what', 'is', 'the', 'chemical', 'symbol', 'for', 'gold', 'au']
Which year did World War II end? 1945
['which', 'year', 'did', 'world', 'war', 'ii', 'end', '1945']
What is the longe

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [48]:
len(vocab)

324

In [49]:
# convert words to numerical indices

def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [50]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [51]:
import torch
from torch.utils.data import Dataset, DataLoader

In [52]:
class QADataset(Dataset):
  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [53]:
dataset = QADataset(df, vocab)

In [54]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [70]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[ 42, 137, 118,   3, 247,   5, 248]]) tensor([249])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([273])
tensor([[ 10,  75,   3, 296,  19, 297]]) tensor([298])
tensor([[  1,   2,   3,  37,  38,  39, 161]]) tensor([162])
tensor([[42, 18,  2, 62, 63,  3, 64, 18]]) tensor([65])
tensor([[ 1,  2,  3, 69,  5, 53]]) tensor([260])
tensor([[ 10, 140,   3, 141, 142,  12, 143,  83,   3, 144]]) tensor([145])
tensor([[  1,   2,   3,   4,   5, 236, 237]]) tensor([238])
tensor([[  1,   2,   3, 234,   5, 235]]) tensor([131])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([106])
tensor([[  1,   2,   3,  37, 133,   5,  26]]) tensor([134])
tensor([[  1,   2,   3,  92, 137,  19,   3,  45]]) tensor([185])
tensor([[ 78,  79, 150, 151,  14, 152, 153]]) tensor([154])
tensor([[ 10,  11, 189, 158, 190]]) tensor([191])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([128])
tensor([[ 1,  2,  3, 33, 34,  5, 35]]) tensor([36])
tensor([[ 78,  79, 261, 151,  14, 262,

In [59]:
import torch.nn as nn

In [75]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50) # converts every word into 50dim layer
    self.rnn = nn.RNN(50,64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output


In [76]:
# Example for how the above RNN code will work
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))
print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [105]:
learning_rate = 0.001
epochs = 20

In [106]:
model = SimpleRNN(len(vocab))

In [107]:
#loss
criterion = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [108]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    #clearinng gradients
    optimizer.zero_grad()

    #forwrd pass
    output = model(question)

    #loss -> output shape (1, 324) - (1)
    loss = criterion(output, answer[0])

    #gradients
    loss.backward()

    #update optimizer
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 527.585453
Epoch: 2, Loss: 461.156694
Epoch: 3, Loss: 381.740951
Epoch: 4, Loss: 317.252915
Epoch: 5, Loss: 265.763123
Epoch: 6, Loss: 218.518067
Epoch: 7, Loss: 174.861551
Epoch: 8, Loss: 136.530901
Epoch: 9, Loss: 105.670654
Epoch: 10, Loss: 80.667342
Epoch: 11, Loss: 62.037912
Epoch: 12, Loss: 48.304655
Epoch: 13, Loss: 37.981461
Epoch: 14, Loss: 30.465882
Epoch: 15, Loss: 24.922462
Epoch: 16, Loss: 20.454812
Epoch: 17, Loss: 17.055655
Epoch: 18, Loss: 14.414299
Epoch: 19, Loss: 12.418106
Epoch: 20, Loss: 10.736389


In [109]:
def predict(model, question, threshold=0.5):

  # convert question to answers
  numerical_question = text_to_indices(question, vocab)

  #tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probabilities
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I dont know")

  print(list(vocab.keys())[index])

In [110]:
predict(model, "What is the largest planet in our solar system?")

jupiter
